# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'capabilities page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'capabilities page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'social media', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social media', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social media',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
select_relevant_links("https://www.proitbridge.com")

{'links': [{'type': 'about page',
   'url': 'https://www.proitbridge.com/about-us'},
  {'type': 'careers page', 'url': 'https://www.proitbridge.com/careers/'},
  {'type': 'contact page', 'url': 'https://www.proitbridge.com/contact-us/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/data-science-ai-program/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/1_courses-data-analytics-program/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/3_courses-it-corporate-training/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/4_courses-college-guest-lectures/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/5_courses-data-science-and-ai-consulting-services/'},
  {'type': 'training program page',
   'url': 'https://www.proitbridge.com/6_courses-11-personalized-career-consultation/'},
  {'type': 'interview preparation kit',
   'url': 'https://www.proitbr

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'credentials page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3-Coder-Next
Updated
7 days ago
•
141k
•
680
zai-org/GLM-OCR
Updated
1 day ago
•
373k
•
905
openbmb/MiniCPM-o-4_5
Updated
1 day ago
•
30.4k
•
664
ACE-Step/Ace-Step1.5
Updated
7 days ago
•
28.7k
•
487
moonshotai/Kimi-K2.5
Updated
5 days ago
•
504k
•
1.95k
Browse 2M+ models
Spaces
Running
on
A100
Featured
239
ACE-Step v1.5
🎵
239
Music Generation Foundation Model v1.5
Running
on
Zero
Featured
1.34k
Qwen3-TTS Demo
🎙
1.34k
Generate speech with custom, cloned, or preset voices
Running
535
Demo Playground
⚡
535
Free platform to acc

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-Coder-Next\nUpdated\n7 days ago\n•\n141k\n•\n680\nzai-org/GLM-OCR\nUpdated\n1 day ago\n•\n373k\n•\n905\nopenbmb/MiniCPM-o-4_5\nUpdated\n1 day ago\n•\n30.4k\n•\n664\nACE-Step/Ace-Step1.5\nUpdated\n7 days ago\n•\n28.7k\n•\n487\nmoonshotai/Kimi-K2.5\nUpdated\n5 days ago\n•\n504k\n•\n1.95k\nBrowse 2M+ models\nSpaces\nRunning\non\nA100\nFeatured\n239\nACE-Step v1.5\n🎵\n239\nMusic Gene

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a pioneering AI community and collaboration platform dedicated to building the future of machine learning (ML). It serves as a central hub where ML engineers, scientists, and enthusiasts globally create, share, and collaborate on state-of-the-art ML models, datasets, and applications across various data modalities such as text, image, video, audio, and even 3D.

The platform hosts over **2 million models**, **500,000 datasets**, and **over 1 million applications**, making it the largest open-source ecosystem empowering the next generation of AI development and innovation. Hugging Face is committed to building an open, ethical, and inclusive AI future through community-driven collaboration.

---

## What They Offer

### The Platform
- **Model Hub:** Access, share, and collaborate on over 2 million open-source machine learning models.
- **Datasets Hub:** Explore and contribute to over 500,000 datasets designed for ML training and benchmarking.
- **Spaces:** Deploy and experiment with interactive AI applications powered by top models, supporting seamless sharing and usage.
- **Multi-Modality Support:** Work with various data types including text, images, video, audio, and 3D.

### Collaboration & Open Source
- Unlimited public hosting of ML models, datasets, and applications.
- Tools to build your ML portfolio and showcase your work to the global AI community.
- Open source software stack ensuring freedom, innovation, and rapid progress.

### Enterprise Solutions
- Secure, scalable AI platform tailored for teams and businesses.
- Features include enterprise-grade security, access controls, and dedicated support.
- Paid Compute options to accelerate ML workloads efficiently.

---

## Company Culture

Hugging Face fosters an open, ethical, and collaborative environment focused on:
- **Community-driven AI:** Empowering researchers, developers, and users to share their work and learn from each other.
- **Transparency and openness:** Providing free access to a vast range of resources, models, and tools.
- **Innovation:** Supporting rapid development of next-gen AI technologies in a nurturing and inclusive ecosystem.

---

## Customers & Use Cases

Hugging Face serves a diverse customer base including:
- Machine Learning researchers advancing AI science
- Developers and data scientists building AI-powered applications
- Enterprises seeking secure, scalable AI solutions for their teams
- Educational institutions fostering AI learning and projects
- AI enthusiasts wanting to explore and contribute to cutting-edge tools and models

---

## Careers and Opportunities

Join Hugging Face to be at the forefront of AI innovation. The company offers:
- Opportunities to work on open-source projects impacting millions worldwide.
- Collaborative environment with passionate, diverse AI experts.
- Roles ranging from ML engineering, research, software development, to enterprise solutions and community growth.
- Chance to contribute to a mission of building an ethical, accessible AI future.

---

## Contact & Get Started

- Visit: [huggingface.co](https://huggingface.co)
- Sign up for free to explore models, datasets, and AI applications.
- Discover enterprise plans for team collaboration and compute needs.
- Join the fast-growing AI community shaping the future of machine learning.

---

**Hugging Face** — Empowering the AI community to innovate, collaborate, and create the future of intelligent technology.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future. It is the premier platform where the machine learning community collaborates on models, datasets, and applications across multiple AI modalities including text, image, video, audio, and even 3D.  
With over **2 million models** and **500,000 datasets**, Hugging Face empowers individuals and organizations to create, discover, and share cutting-edge machine learning projects worldwide.

---

## What We Offer

- **Models & Datasets:** Access and contribute to a vast repository of public models and datasets for all AI needs.
- **Spaces:** Host and deploy AI applications easily with customizable environments running on powerful hardware like A100 GPUs.
- **Community Collaboration:** Build your machine learning profile, share your work, and join a vibrant ecosystem of AI developers and researchers.
- **Multi-Modality:** Explore AI applications across various data types — text, images, audio, video, and more.
- **Open Source Stack:** Utilize a robust open source infrastructure designed to accelerate AI development and deployment.

---

## Enterprise & Team Solutions

Hugging Face provides advanced solutions tailored for businesses and organizations:

- **Team Plan:** Starting at $20/user/month. Ideal for growing teams to collaborate seamlessly with instant setup.
- **Enterprise Plan:** Flexible contract options with security and compliance features.
  
### Enterprise Features:

- **Enterprise-grade security:** Single Sign-On (SSO), granular access control with resource groups, organization-wide security policies.
- **Data Management:** Control repository data locations by region, private dataset viewer, private storage extra capacity (1TB per member).
- **Advanced Compute:** Boost scalability and performance with options like ZeroGPU quota boost for fast inference.
- **Analytical Insights:** Centralized analytics dashboard for tracking usage and spending.
- **Managed Billing & Budget Control:** Yearly commitment and usage monitoring tools.
- **Priority Technical Support:** Dedicated help to maximize platform usage.

Join many forward-thinking AI organizations who rely on Hugging Face to power their AI innovation in a secure and scalable environment.

---

## Pricing Overview

- **Pro Account:** $9/month  
  - Enhanced private storage (10x), more inference credits (20x), highest queue priority, Spaces Dev Mode, and publishing capabilities on your HF profile.
- **Team Account:** $20/user/month  
  - Includes all Pro features plus SSO and collaboration tools for teams.

---

## Company Culture

Hugging Face fosters an open, collaborative, and inclusive community culture. They emphasize sharing knowledge, growing together, and pushing the boundaries of AI technology in ethical and impactful ways. Their community spirit invites everyone from hobbyists to experts to contribute and learn, building a diverse and supportive environment.

---

## Careers & Opportunities

Hugging Face is continuously growing and looking for talented individuals passionate about AI, open source, and community-building. Working at Hugging Face means joining a forward-thinking company where innovation, collaboration, and impact are at the core. 

If you want to be part of shaping the future of AI and work alongside leading experts in machine learning, Hugging Face offers exciting roles in engineering, data science, research, product management, and more.

---

## Join Us

Explore AI, collaborate globally, and build the future with Hugging Face:

- Create your free account and access millions of models and datasets.
- Upgrade to Pro for enhanced features or scale with Team and Enterprise plans.
- Participate in the vibrant AI community driving progress through open collaboration.

**Website:** [https://huggingface.co](https://huggingface.co)

---

### Tags: AI community, Machine Learning, Open Source, Models, Datasets, Enterprise AI, Collaboration, Innovation, Careers, AI Applications

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>